# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Zhang  ->  M. Zhang  |  ['M. Zhang']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Scheuck  ->  M. Scheuck  |  ['M. Scheuck']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
P. Boley  ->  P. Boley  |  ['P. Boley']
T. Henning  ->  T. Henning  |  ['T. Henning']


D. Semenov  ->  D. Semenov  |  ['D. Semenov']
Arxiv has 97 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.17900


extracting tarball to tmp_2511.17900... done.
Retrieving document from  https://arxiv.org/e-print/2511.18029


extracting tarball to tmp_2511.18029...

 done.


...

extracting tarball to tmp_2511.18981...

 done.


Retrieving document from  https://arxiv.org/e-print/2511.19148


extracting tarball to tmp_2511.19148... done.


D. Semenov  ->  D. Semenov  |  ['D. Semenov']


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.18029-b31b1b.svg)](https://arxiv.org/abs/2511.18029) | **ACES: The Magnetic Field in Large Filaments in the Galactic Center**  |
|| D. M. Paré, et al. -- incl., <mark>P. Garcia</mark> |
|*Appeared on*| *2025-11-25*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            The Galactic Center (GC) is an extreme region of the Milky Way that is host to a complex set of thermal and non-thermal structures. In particular, the GC contains high-density gas and dust that is collectively referred to as the Central Molecular Zone (CMZ). In this work, we study a subset of HNCO filaments identified in band 3 ALMA observations of the GC obtained by the ALMA CMZ Exploration Survey (ACES) that are comparable to high density filaments identified in the Galactic Disk. We compare the orientation of the magnetic field derived from 214 um SOFIA and 850 um JCMT observations with the filament orientation to determine which mechanisms dominate the formation of these filaments. We observe a large range of magnetic orientations in our observed filaments indicating the complex environments the filaments are located in. We also compare the observational results to synthetic data sets created using an MHD model of the GC. Our analysis reveals that the dominant mechanisms local to the HNCO filaments vary throughout the GC with some filaments being dominated by supersonic turbulence and others by subsonic turbulence. The comparison to synthetic observations indicates that the observed filaments are in magnetically dominated environments that could be supporting these filaments against collapse. Our results on the CMZ filaments are also compared to results obtained on similar filaments located in the Galactic Disk, and we find that the filaments studied here are possible CMZ analogs to the dense filamentary "bones" observed previously in the Galactic Disk.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.18981-b31b1b.svg)](https://arxiv.org/abs/2511.18981) | **The complex inner disk of the Herbig Ae star HD 100453 with VLTI/MATISSE**  |
|| L. N. A. v. Haastere, et al. -- incl., <mark>M. Scheuck</mark>, <mark>R. v. Boekel</mark>, <mark>P. Boley</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-11-25*|
|*Comments*| *17 pages, 12 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            The inner regions of planet-forming disks hold invaluable insights for our understanding of planet formation. The disk around the Herbig star HD 100453 presents one such environment, with an inner disk that is significantly misaligned with respect to the outer disk. This paper expands the existing H-band (PIONIER) and K-band (GRAVITY) interferometric studies of the HD 100453 inner disk to the L-band with the MATISSE VLTI instrument. With snapshot data spanning approximately four years we aim for a more comprehensive understanding of the inner disk structures and their potential time evolution. Based on the MATISSE data obtained, we use a combination of analytical models and image reconstruction to constrain the disk structure. Additionally, we fit a temperature gradient model to the selected wavelength range of PIONIER, GRAVITY and MATISSE to derive physical properties of the inner regions. Our parametric model finds an inclination of $\sim 47.5^\circ$ and a position angle of $\sim 83.6^\circ$, which corroborates the case of strong inner-outer disk misalignment. From the symmetric temperature gradient we derive an inner disk radius around $\sim0.27$ au, with dust surface densities of $\Sigma_{\rm{subl}} \approx 10^{-3.2}$ g/cm$^2$ and vertical optical depth $\tau_{\rm{z, subl}} \approx 0.1-0.06$. Same-night MATISSE and GRAVITY observations indicate the necessity for higher-order asymmetries to explain the interferometric signals, which is further supported by a MATISSE snapshot image reconstruction. The chromatic interferometric data reveal the likely need for higher-order asymmetries to explain the inner disk of HD~100453, suggesting a possible origin in dynamic interactions or disk instabilities. Coordinated multi-wavelength infrared interferometric observations with GRAVITY and MATISSE will be crucial to confirm these findings and uncover its underlying nature.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.19148-b31b1b.svg)](https://arxiv.org/abs/2511.19148) | **Observations of sulfuretted species in HL Tau**  |
|| P. Rivière-Marichalar, et al. -- incl., <mark>D. Semenov</mark> |
|*Appeared on*| *2025-11-25*|
|*Comments*| **|
|**Abstract**|            Protoplanetary disks inherit their chemical composition from their natal molecular cloud, but the extent to which this material is preserved versus reset through chemical reprocessing remains an open question. Understanding this balance is a major topic in astrochemistry. Comparing the chemical composition of the envelope and the protoplanetary disk is key to solving the topic. The goal of this paper is to investigate the chemical differences between the disk and the surrounding envelope by comparing the column density ratios of a few selected species in each region. The source we focus on is HL Tau. We present new NOEMA observations of HL Tau targeting the following species: CS, H2CO, H2S, and SO2. We produced zeroth-, first-, and second-moment maps for the species where emission was detected and used them to analyze the spatial distribution and kinematic properties of the different molecules in the disk and the envelope. We derived the column densities and compared the values derived for the envelope and disk. We also computed the rotational diagram for the SO2 detected transitions. Assuming two different temperature regimes, 17 and 58 K, we derived column densities for the species surveyed in the disk and compared them with values derived for the envelope. We find large differences in the derived column density ratios of the surveyed molecules, especially for N(CS)/N(H2S), which is 40 to 50 times larger in the envelope. We attribute these variations to the different excitation and UV-irradiation regimes in the disk and envelope. We also note strong gradients in the ratios between different positions of the disk and tentatively attribute them to different levels of turbulence at different azimuths. The observed differences in molecular ratios in the envelope and the disk are suggestive of chemical reprocessing of the gas during the formation and evolution of the protoplanetary disk.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.17900-b31b1b.svg)](https://arxiv.org/abs/2511.17900) | **Revisiting $γ$-Ray Orbital Modulation in the Redback Millisecond Pulsar PSR J2039-5617**  |
|| <mark>M. Zhang</mark>, S. Pei, S. Chang, P. Zhang |
|*Appeared on*| *2025-11-25*|
|*Comments*| *9 pages, 6 figures, accepted by ApJ*|
|**Abstract**|            PSR J2039-5617 is a redback millisecond pulsar binary system consisting of a compact star with a mass of 1.1-1.6 $M_\odot$ and a low-mass companion of 0.15-0.22 $M_\odot$. For this binary, we performed a timing analysis using 16 years of data from the Fermi Large Area Telescope, covering the period from 2008 August to 2024 October. Our analysis detected an orbital modulation with a period of 0.2279781 days at a significance level of $\sim4\sigma$, which is in good agreement with previous findings. However, unlike previous reports, we identified a transition in the orbital modulation around 2021 August, after which the orbital signal disappeared. We speculate that the system may be undergoing a transition from a rotation-powered to an accretion-powered state at this epoch. Additionally, we conducted the phase-resolved and spectral analyses, and in the phase-resolved results, we observed an anti-correlation between its $\gamma$-ray and X-ray emissions, which consistent with the predictions of high-energy radiation models for such systems. We provide some predictive discussions based on the results of $\gamma$-ray data analysis, and future Fermi-LAT observations will determine whether these predictions hold true.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.18478-b31b1b.svg)](https://arxiv.org/abs/2511.18478) | **Prospects for cosmological research using hundred-meter-class radio telescopes: 21-cm intensity mapping survey strategies with QTT, JRT, and HRT**  |
|| J.-D. Pan, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-11-25*|
|*Comments*| *7 pages, 5 figures*|
|**Abstract**|            Understanding dark energy requires precision measurements of the expansion history of the universe and the growth of large-scale structure. The 21 cm intensity mapping (21 cm IM) technique enables rapid large-area surveys that can deliver these measurements. China is constructing three hundred-meter-class single-dish radio telescopes, including the QiTai 110 m Radio Telescope (QTT), the 120 m Jingdong Radio Telescope (JRT), and the 120 m Huadian Radio Telescope (HRT), whose designs are well suited for 21 cm IM cosmology. We use a Fisher-to-MCMC forecasting framework to evaluate the baryon acoustic oscillations / redshift space distortions (BAO/RSD) measurement capabilities of QTT, JRT, and HRT and propagate them to dark-energy constraints in the $w_0w_a$CDM model. Our results show that achieving a redshift coverage up to $z_{\mathrm{max}} = 1$ is crucial for fully realising the potential of hundred-meter-class single-dish telescopes for 21 cm cosmology. If all three telescopes carry out 21 cm IM surveys over the same redshift range up to $z_{\mathrm{max}}=1$ and combine their BAO/RSD measurements, QTT+JRT+HRT yield $\sigma(w_0)=0.094$ and $\sigma(w_a)=0.487$, providing tighter constraints than DESI DR2 results.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.18029/./RO_dist_SOFIA_vs_RHT.png', 'tmp_2511.18029/./AM_SOFIA_VS_RHT_CMZ.png', 'tmp_2511.18029/./Fil_Map.png', 'tmp_2511.18029/./fil2_12as_10.png', 'tmp_2511.18029/./fil4_12as_10.png', 'tmp_2511.18029/./fil5_12as_10.png']
copying  tmp_2511.18029/./RO_dist_SOFIA_vs_RHT.png to _build/html/
copying  tmp_2511.18029/./AM_SOFIA_VS_RHT_CMZ.png to _build/html/
copying  tmp_2511.18029/./Fil_Map.png to _build/html/
copying  tmp_2511.18029/./fil2_12as_10.png to _build/html/
copying  tmp_2511.18029/./fil4_12as_10.png to _build/html/
copying  tmp_2511.18029/./fil5_12as_10.png to _build/html/
exported in  _build/html/2511.18029.md
    + _build/html/tmp_2511.18029/./RO_dist_SOFIA_vs_RHT.png
    + _build/html/tmp_2511.18029/./AM_SOFIA_VS_RHT_CMZ.png
    + _build/html/tmp_2511.18029/./Fil_Map.png
    + _build/html/tmp_2511.18029/./fil2_12as_10.png
    + _build/html/tmp_2511.18029/./fil4_12as_10.png
    + _build/html/tmp_2511.18029/./fil5_12as_10.png
found figures ['tmp_2511.189

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\Mjsr}{MJy sr^{-1}}$
$\newcommand{\kmsp}{km s^{-1} pc^{-1}}$
$\newcommand{\ergs}{ergs s^{-1}}$
$\newcommand{\ergy}{ergs yr^{-1}}$
$\newcommand{\jyb}{Jy b^{-1}}$
$\newcommand{\jybe}{Jy beam^{-1}}$
$\newcommand{\hii}{H~{\scriptsize II}}$
$\newcommand{\cm}{cm^{-3}}$
$\newcommand{\cms}{cm^{-2}}$
$\newcommand{\mpc}{masers pc^{-2}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\msuny}{M_{\odot} yr^{-1}}$
$\newcommand{\mdot}{\dot\textrm{M}}$
$\newcommand{\pdot}{\dot\textrm{p}}$
$\newcommand{\edot}{\dot\textrm{E}}$
$\newcommand{\til}{\sim}$
$\newcommand{\ring}{M0.8--0.2}$
$\newcommand{\ie}{i.e.}$
$\newcommand{\eg}{e.g.}$
$\newcommand{\ratio}{\langle B_{t}^{2} \rangle/\langle B_{0}^{2} \rangle}$
$\newcommand{\ratiof}{\frac{\langle B_{t}^{2} \rangle}{\langle B_{0}^{2} \rangle}}$
$\newcommand{\dispfunct}{1 - \langle\cos[\Delta \phi(\ell)] \rangle}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\dtc}[1]{\textcolor{red}{(DTC: #1)}}$
$\newcommand{\kk}[1]{\textcolor{violet}{(KK: #1)}}$</div>



<div id="title">

# $\uppercase{ACES: The Magnetic Field in Large Filaments in the Galactic Center}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.18029-b31b1b.svg)](https://arxiv.org/abs/2511.18029)<mark>Appeared on: 2025-11-25</mark> -  _Accepted for publication in ApJ_

</div>
<div id="authors">

D. M. Paré, et al. -- incl., <mark>P. Garcia</mark>

</div>
<div id="abstract">

**Abstract:** The Galactic Center (GC) is an extreme region of the Milky Way that is host to a complex set of thermal and non-thermal structures. In particular, the GC contains high-density gas and dust that is collectively referred to as the Central Molecular Zone (CMZ). In this work, we study a subset of HNCO filaments identified in band 3 ALMA observations of the GC obtained by the ALMA CMZ Exploration Survey (ACES) that are comparable to high density filaments identified in the Galactic Disk. We compare the orientation of the magnetic field derived from 214 $\mu$ m SOFIA and 850 $\mu$ m JCMT observations with the filament orientation to deftermine which mechanisms dominate the formation of these filaments. We observe a large range of magnetic orientations in our observed filaments indicating the complex environments the filaments are located in. We also compare the observational results to synthetic data sets created using an MHD model of the GC. Our analysis reveals that the dominant mechanisms local to the HNCO filaments vary throughout the GC with some filaments being dominated by supersonic turbulence and others by subsonic turbulence. The comparison to synthetic observations indicates that the observed filaments are in magnetically dominated environments that could be supporting these filaments against collapse. Our results on the CMZ filaments are also compared to results obtained on similar filaments located in the Galactic Disk, and we find that the filaments studied here are possible CMZ analogs to the dense filamentary "bones" observed previously in the Galactic Disk.

</div>

<div id="div_fig1">

<img src="tmp_2511.18029/./RO_dist_SOFIA_vs_RHT.png" alt="Fig8.1" width="50%"/><img src="tmp_2511.18029/./AM_SOFIA_VS_RHT_CMZ.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** The relative orientation (left panel) and alignment measure (right panel) distributions obtained for the full set of targeted LFs studied using the 214 $\mu$m FIREPLACE observations. In the left panel, dashed and dotted lines show projected relative angles for intrinsic 3D angles of 20◦ee to 60◦ee and 75◦ee to 90◦ee, respectively. The red histogram is a 7:3 mix of these distributions, matching the bimodal pattern in the data. (*fig:RO_dist_SOFIA_RHT*)

</div>
<div id="div_fig2">

<img src="tmp_2511.18029/./Fil_Map.png" alt="Fig5" width="100%"/>

**Figure 5. -** The final RHT distribution used to identify filamentary structures in the ACES HNCO distribution. We show the RHT distribution obtained from the HNCO distribution that has been convolved to the 19.6$\arcsec$ FIREPLACE beam size. The black contour traces the cool dust traced by the FIREPLACE 214 $\mu$m observations at a level of 6 Jy pixel$^{-1}$ and indicates the CMZ molecular clouds that have been masked out. The boxes indicate the locations of the filaments that are targeted for a detailed analysis in this work. Blue boxes indicate filaments that coincide with  only FIREPLACE magnetic field orientations and green boxes indicate filaments that coincide with both FIREPLACE and BISTRO magnetic field orientations. In addition, red and orange boxes indicate the filaments previously studied in Battersby et al. (submitted) where the red boxes coincide with only FIREPLACE magnetic field orientations and the orange box coincides with both FIREPLACE and BISTRO magnetic field orientations. The filaments studied within each of these regions are shown in Figures \ref{fig:hro_rht_sofia_fig1} and \ref{fig:hro_rht_sofia_fig2}. (*fig:fil_map*)

</div>
<div id="div_fig3">

<img src="tmp_2511.18029/./fil2_12as_10.png" alt="Fig11.1" width="33%"/><img src="tmp_2511.18029/./fil4_12as_10.png" alt="Fig11.2" width="33%"/><img src="tmp_2511.18029/./fil5_12as_10.png" alt="Fig11.3" width="33%"/>

**Figure 11. -** Observed relative orientation and alignment measure (AM) results obtained for individual LFs compared to the 12$\arcsec$ BISTRO (850 $\mu$m) magnetic field in the CMZ located in regions 2, 4, and 5. Left column: the HNCO intensity of the LFs with the non-filamentary emission surrounding the LFs masked out. Thicker red dashes indicate the BISTRO magnetic field vectors used to calculate the AM values. Middle column: the histogram or relative orientations between the BISTRO magnetic field orientation and the LF orientation. Right column: the AM measure results obtained for each LF studied. (*fig:hro_rht_jcmt*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.18029"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cmark}{\checkmark}$
$\newcommand{\xmark}{--}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$</div>



<div id="title">

# The complex inner disk of the $\mbox{Herbig Ae}$ star HD 100453 with VLTI/MATISSE

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.18981-b31b1b.svg)](https://arxiv.org/abs/2511.18981)<mark>Appeared on: 2025-11-25</mark> -  _17 pages, 12 figures, accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

L. N. A. v. Haastere, et al. -- incl., <mark>M. Scheuck</mark>, <mark>R. v. Boekel</mark>, <mark>P. Boley</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** The inner regions of planet-forming disks hold invaluable insights for our understanding of planet formation. Of particular interest are inner disk regions that may be influenced by already formed planets. The disk around the Herbig star HD 100453 presents one such environment, with an inner disk that is significantly misaligned with respect to the outer disk. This paper expands the existing H-band (PIONIER) and K-band (GRAVITY) interferometric studies of the HD 100453 inner disk to the L-band with the MATISSE VLTI instrument. With snapshot data spanning approximately four years we aim for a more comprehensive understanding of the inner disk structures and their potential time evolution. Based on the MATISSE data obtained, we use a combination of analytical models and image reconstruction to constrain the disk structure. Additionally, we fit a temperature gradient model to the selected wavelength range of PIONIER, GRAVITY and MATISSE to derive physical properties of the inner regions. Our parametric model finds an inclination of $\sim 47.5^\circ$ and a position angle of $\sim 83.6^\circ$ , which corroborates the case of strong inner-outer disk misalignment. From the symmetric temperature gradient we derive an inner disk radius around $\sim0.27$ au, with dust surface densities of $\Sigma_{\rm{subl}} \approx 10^{-3.2}$ g/cm $^2$ and vertical optical depth $\tau_{\rm{z, subl}} \approx 0.1-0.06 $ .   Same-night MATISSE and GRAVITY observations show directional discrepancies not consistent with a first-order azimuthally modulation ring, indicating the necessity for higher-order asymmetries to explain the interferometric signals. This interpretation is further supported by a MATISSE snapshot image reconstruction, which reveals a two-component asymmetric structure. The chromatic interferometric data reveal the likely need for higher-order asymmetries to explain the inner disk of HD 100453, suggesting a possible origin in dynamic interactions or disk instabilities. Coordinated multi-wavelength infrared interferometric observations with GRAVITY and MATISSE will be crucial to confirm these findings and uncover its underlying nature.

</div>

<div id="div_fig1">

<img src="tmp_2511.18981/./updated_figures/fig2_observations_overview.png" alt="Fig5" width="100%"/>

**Figure 5. -** Calibrated squared visibilities and closure phases from all the selected MATISSE observations.
            (*Fig:overview_data_all*)

</div>
<div id="div_fig2">

<img src="tmp_2511.18981/./updated_figures/fig4_matisse_skwpa_timeplot.png" alt="Fig2" width="100%"/>

**Figure 2. -** Violin-plot showcasing the time-variable MCMC fitting results of the asymmetry angle ($\theta_{\rm{skwPA}}$) for six epochs of MATISSE data, where the other disk parameters are fixed to the best-fit model-two asymmetric disk parameters from Table \ref{tab:model_fit_onlymatisse}. The horizontal dashed line shows the best fitted value for the asymmetry angle for the time-invariant dataset. Above each violin the change in the respective $\chi^2_r$ is shown compared to the time-invariant value. The error bars indicate the best fit value with 16/84\% percentile intervals.
            (*Fig:Time_Variability*)

</div>
<div id="div_fig3">

<img src="tmp_2511.18981/./updated_figures/fig7_gravity_matisse_skwpa_timevariability.png" alt="Fig3" width="100%"/>

**Figure 3. -**  Similar to Fig. \ref{Fig:Time_Variability}, we show the MCMC results for the asymmetry angle of the first-order skewed ring model using MATISSE data from the 2021ab epoch (orange), alongside re-reduced GRAVITY data (blue) obtained close in time. The horizontal dashed lines indicate the averaged H-, K- and L-band estimate for the direction of semi-major axis of the inner dust ring. In the top-left we indicate the approximate Keplerian period, based on the stellar parameters, the best-fit temperature gradient inner radius $R_{in}$ and  parametric half-light radius $l_a$.
            (*Fig:SKWPA_Time_Gravity_fit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.18981"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Observations of sulfuretted species in HL Tau

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.19148-b31b1b.svg)](https://arxiv.org/abs/2511.19148)<mark>Appeared on: 2025-11-25</mark> - 

</div>
<div id="authors">

P. Rivière-Marichalar, et al. -- incl., <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** Protoplanetary disks inherit their chemical composition from their natal molecular cloud, although the extent to which this material is preserved versus reset through chemical reprocessing remains an open question. Understanding this balance is a major topic in astrochemistry and star and planet formation. Comparing the chemical composition of the envelope and the protoplanetary disk is key to solving the topic. However, disentangling protoplanetary disk emission from envelope emission is not an easy task. The goal of this paper is to investigate the chemical differences between the disk and the surrounding envelope by comparing the column density ratios of a few selected species in each region. The source we focus on is HL Tau, where molecular absorption lines from the envelope have been detected, thus allowing for the derivation of column densities and molecular abundances. We present new NOEMA observations of HL Tau targeting the following species: CS, H $\rm _2$ CO,  H $\rm _2$ S, and SO $\rm _2$ . We produced zeroth-, first-, and second-moment maps for the species where emission was detected and used them to analyze the spatial distribution and kinematic properties of the different molecules in the disk and the envelope. We derived the column densities and compared the values derived for the envelope and disk. We also computed the rotational diagram for the SO $\rm _2$ detected transitions. Assuming two different temperature regimes, 17 and 58 K, we derived column densities for the species surveyed in the disk and compared them with values derived for the envelope. We find large differences in the derived column density ratios of the surveyed molecules, especially for N(CS)/N(H $\rm _2$ S), which is 40 to 50 times larger in the envelope. We attribute these variations to the different excitation and UV-irradiation regimes in the disk and envelope. We also note strong gradients in the ratios between different positions of the disk and tentatively attribute them to different levels of turbulence at different azimuths. The observed differences in molecular ratios in the envelope and the disk are suggestive of chemical reprocessing of the gas during the formation and evolution of the protoplanetary disk.

</div>

<div id="div_fig1">

<img src="tmp_2511.19148/./HLTau_SO2_rot_diag.png" alt="Fig5.1" width="50%"/><img src="tmp_2511.19148/./HLTau_SO2_rot_diag_Garufi.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Rotational diagram of SO$\rm _2$ transitions detected with NOEMA. The top panel shows the rotational diagram for the transitions reported in this paper alone, while the bottom panel also includes the detections by [Garufi, et. al (2022)](https://ui.adsabs.harvard.edu/abs/2022A&A...658A.104G) in red.  (*Fig:SO2_rot_diag*)

</div>
<div id="div_fig2">

<img src="tmp_2511.19148/./HLTau_CS_moments_gridplot_smooth.png" alt="Fig9.1" width="33%"/><img src="tmp_2511.19148/./HLTau_H2S_moments_gridplot_smooth.png" alt="Fig9.2" width="33%"/><img src="tmp_2511.19148/./HLTau_H2CO_moments_gridplot_smooth.png" alt="Fig9.3" width="33%"/>

**Figure 9. -** From left to right: Continuum-subtracted zeroth-, first-, and second-moment map of CS 3-2 (top), H$\rm _2$S   1$\rm _{10}$-1$\rm _{01}$(middle), and H$\rm _2$CO $\rm 5_6-4_5$(bottom) observed with NOEMA. Before the moment calculation, a four-sigma clipping mask was applied to the data cubes. The cubes were smoothed by a factor of three before computing the moment maps. Only channels in the range 3 to 11 km s$\rm ^{-1}$ and with a S/N larger than four have been used. (*Fig:moments*)

</div>
<div id="div_fig3">

<img src="tmp_2511.19148/./HLTau_moments_KeplerMask.png" alt="Fig10" width="100%"/>

**Figure 10. -** From left to right: Integrated intensity maps of CS 3-2 (left), H$\rm _2$S 1$\rm _{10}$-1$\rm _{01}$(middle), and H$\rm _2$CO $\rm 5_6-4_5$(right) after a Keplerian mask has been applied. The black star in each map marks the position of the emission peak of the map. (*Fig:moments_KeplerMask*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.19148"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

133  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
